In [2]:
import torch
import logging


In [3]:
tensor0d = torch.tensor(1)
print(tensor0d)
print(tensor0d.shape)
print(tensor0d.dtype)

tensor(1)
torch.Size([])
torch.int64


In [4]:
tensor1d = torch.tensor([1,2])
print(tensor1d)
print(tensor1d.shape)
print(tensor1d.dtype)

tensor([1, 2])
torch.Size([2])
torch.int64


In [5]:
tensor1d = torch.tensor([1.1,2])
print(tensor1d)
print(tensor1d.shape)
print(tensor1d.dtype)

tensor([1.1000, 2.0000])
torch.Size([2])
torch.float32


In [6]:
tensor2d = torch.tensor([[1,2], [11,22]])
print(tensor2d)
print(tensor2d.shape)
print(tensor2d.dtype)

tensor([[ 1,  2],
        [11, 22]])
torch.Size([2, 2])
torch.int64


In [7]:
tensor3d = torch.tensor([[[1, 2], [3,4]], [[43, 44], [54, 55]]])
print(tensor3d)
print(tensor3d.shape)
print(tensor3d.dtype)

tensor([[[ 1,  2],
         [ 3,  4]],

        [[43, 44],
         [54, 55]]])
torch.Size([2, 2, 2])
torch.int64


In [8]:
tensor2d = torch.tensor([[1, 2], [3,4]])
print(tensor2d)
print(tensor2d.shape)
tensor2d @ tensor2d.T

tensor([[1, 2],
        [3, 4]])
torch.Size([2, 2])


tensor([[ 5, 11],
        [11, 25]])

In [9]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1= torch.tensor([2.2], requires_grad = True)
b = torch.tensor([0.0], requires_grad=True)

z=x1 * w1 + b

a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)
print(loss)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)


In [10]:
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [11]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),

        )

    
    def forward(self, x):
        logits = self.layers(x)
        return logits


In [12]:
model = NeuralNetwork(50, 3)

In [13]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"trainable parameters are {num_params}")

trainable parameters are 2213


In [15]:
print(model.layers[0])

Linear(in_features=50, out_features=30, bias=True)


In [16]:
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.1093, -0.0321, -0.1259,  ...,  0.0739,  0.0303, -0.0194],
        [ 0.0421,  0.0501,  0.0089,  ...,  0.0961, -0.0880,  0.0944],
        [ 0.0637,  0.0969, -0.1160,  ..., -0.0254, -0.0649,  0.0936],
        ...,
        [ 0.1278, -0.0850,  0.0337,  ..., -0.0048,  0.1104, -0.0576],
        [-0.0233,  0.1235, -0.1351,  ..., -0.0371, -0.0630, -0.0567],
        [-0.0889, -0.0870, -0.0416,  ...,  0.0052,  0.0381, -0.0137]],
       requires_grad=True)


In [17]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [18]:
# to make the outputs reproducible
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [21]:
torch.manual_seed(123)
X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [22]:
# while inferencing to get the calculation without grads
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [23]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


In [24]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])
 
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [25]:
print(X_train)
print(y_train)
print(X_test)
print(y_test)

tensor([[-1.2000,  3.1000],
        [-0.9000,  2.9000],
        [-0.5000,  2.6000],
        [ 2.3000, -1.1000],
        [ 2.7000, -1.5000]])
tensor([0, 0, 0, 1, 1])
tensor([[-0.8000,  2.8000],
        [ 2.6000, -1.6000]])
tensor([0, 1])


In [26]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y


    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    

    def __len__(self):
        return self.labels.shape[0]
    

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [27]:
print(len(train_ds))

5


In [32]:
from torch.utils.data import DataLoader


torch.manual_seed(123)


train_loader = DataLoader(

    dataset=train_ds,
    batch_size = 2,
    shuffle=True,
    num_workers = 0,
)


test_loader = DataLoader(
    dataset = test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
)

In [33]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [34]:
train_loader = DataLoader(

    dataset=train_ds,
    batch_size = 2,
    shuffle=True,
    num_workers = 0,
    drop_last=True,
)

In [35]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


In [42]:
# Typical training loop
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### Logging
        print(f"Epoch {epoch+1:03d}/{num_epochs:03d}"
              f"| Batch {batch_idx:03d}/{len(train_loader):03d}"
              f"| Train loss: {loss:.2f}")
        
    model.eval()

Epoch 001/003| Batch 000/002| Train loss: 0.75
Epoch 001/003| Batch 001/002| Train loss: 0.65
Epoch 002/003| Batch 000/002| Train loss: 0.44
Epoch 002/003| Batch 001/002| Train loss: 0.13
Epoch 003/003| Batch 000/002| Train loss: 0.03
Epoch 003/003| Batch 001/002| Train loss: 0.00
